## 1. 環境確認

In [ ]:
# GPU確認
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")

## 2. Google Drive マウント

## 4. Gitクローン

In [ ]:
# プロジェクトディレクトリの設定
import os

# ★ここを自分のリポジトリURLに変更★
GITHUB_REPO = "https://github.com/YOUR_USERNAME/atma_22_ca.git"
PROJECT_NAME = "atma_22_ca"

# Git clone（初回のみ）
if not os.path.exists(f"/content/{PROJECT_NAME}"):
    !git clone {GITHUB_REPO} /content/{PROJECT_NAME}
    print(f"✅ Cloned {GITHUB_REPO}")
else:
    print(f"⚠️ {PROJECT_NAME} already exists. Pulling latest changes...")
    !cd /content/{PROJECT_NAME} && git pull

# プロジェクトディレクトリに移動
%cd /content/{PROJECT_NAME}

## 2. パッケージインストール

In [ ]:
# 必要パッケージのインストール
!pip install -q scikit-learn pandas numpy matplotlib seaborn tqdm

## 3. rclone セットアップ（初回のみ）

In [ ]:
# rcloneインストール
!curl https://rclone.org/install.sh | sudo bash

# バージョン確認
!rclone version

In [ ]:
# rclone設定（初回のみ実行）
# 以下のコマンドを実行して、Google Driveを設定してください
# remote名は "gdrive" を推奨

!rclone config

## 5. データ同期（rclone）

In [ ]:
# ★Google Driveのパスを設定★
GDRIVE_DATA_PATH = "gdrive:atma22/data"

# データをダウンロード（初回のみ / 更新があった場合のみ同期）
!rclone sync {GDRIVE_DATA_PATH} /content/{PROJECT_NAME}/data --progress

print("\n✅ Data sync completed")

# データ確認
!ls -lh data/raw/input/atmaCup22_metadata/

## 6. モジュールインポート

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# プロジェクトルートをパスに追加
sys.path.append('/content/atma_22_ca')

from configs.config import *
from src.runner import Runner
from src.model_resnet_knn import ModelResNet50KNN
from src.util import Logger, Util, Metric

print(f"Input dir: {DIR_INPUT}")
print(f"Model dir: {DIR_MODEL}")
print(f"Submission dir: {DIR_SUBMISSIONS}")

## 7. ロガー設定

In [ ]:
logger = Logger(path=DIR_LOG)

def get_run_name(model_type):
    run_name = model_type
    suffix = '_' + datetime.now().strftime("%Y%m%d%H%M")
    run_name = run_name + suffix
    return run_name

logger.info("Logger initialized on Colab")

## 8. データ読み込み

In [ ]:
df_train = pd.read_csv(os.path.join(DIR_INPUT, 'atmaCup22_metadata', 'train_meta.csv'))
df_test = pd.read_csv(os.path.join(DIR_INPUT, 'atmaCup22_metadata', 'test_meta.csv'))

print(f"Train: {df_train.shape}")
print(f"Test: {df_test.shape}")

# グループ化
df_train['group'] = df_train['quarter'] + '_' + df_train['session'].astype(str)

print(f"\n【CV Strategy】")
print(f"  Group column: 'group' (quarter_session)")
print(f"  Total groups: {df_train['group'].nunique()}")
print(f"  Avg samples per group: {len(df_train) / df_train['group'].nunique():.1f}")

## 9. パラメータ設定（GPU最適化）

In [ ]:
memo = "ResNet50 + KNN on Colab GPU"
run_name = get_run_name("resnet50_knn_colab")

print(f"Run name: {run_name}")
print(f"Memo: {memo}")

In [ ]:
# GPU最適化パラメータ
params = {
    'model_name': 'resnet50',
    'k': 5,
    'threshold': 0.5,
    'min2_threshold': 0.3,
    'batch_size': 64,  # GPUならバッチサイズ大きめ
    'num_workers': 2,  # Colabでは2推奨
    'use_cache': True,
}

print("\n=== KNN Parameters (GPU optimized) ===")
print(f"  k: {params['k']}")
print(f"  batch_size: {params['batch_size']} (GPU)")
print(f"  num_workers: {params['num_workers']}")

## 10. CV設定

In [ ]:
cv_strategy = {
    'method': 'GroupKFold',
    'n_splits': 5,
    'shuffle': False,
    'random_state': 42
}

print("=== CV Strategy ===")
print(f"  Method: {cv_strategy['method']}")
print(f"  N splits: {cv_strategy['n_splits']}")

## 11. Runner作成

In [ ]:
runner = Runner(
    run_name=run_name,
    model_cls=ModelResNet50KNN,
    params=params,
    cv_strategy=cv_strategy,
    logger=logger
)

print(f"Runner created: {run_name}")

## 12. 5-Fold CV学習（GPU高速化）

In [ ]:
%%time
# 5-fold CV学習（GPUで高速化）
scores = runner.train_cv(
    df_train, 
    groups=df_train['group'].values
)

print("\n" + "="*80)
print(f"CV Score (Macro F1): {np.mean(scores):.5f} ± {np.std(scores):.5f}")
print("="*80)

## 13. テストデータ予測

In [ ]:
%%time
pred_test = runner.predict_cv(df_test)

print(f"\nTest predictions shape: {pred_test.shape}")
print(f"\nPrediction distribution:")
print(pred_test['label_id'].value_counts().sort_index())

## 14. 提出ファイル作成

In [ ]:
submission_path = runner.save_submission(pred_test, run_name)

print(f"\n✅ Submission file saved: {submission_path}")
print(f"\nFirst 10 predictions:")
print(pred_test.head(10))

## 15. kチューニング実験（高速）

In [ ]:
k_values = [3, 5, 7, 10, 15]
k_results = []

print("=== Testing different k values ===")

for k in k_values:
    predictions_list = []
    
    for i_fold in range(runner.n_splits):
        model = runner.load_model_cv(i_fold)
        
        if hasattr(model, 'test_similarities') and model.test_similarities is not None:
            pred = model.predict_with_custom_threshold(
                threshold=params['threshold'],
                min2_threshold=params['min2_threshold'],
                k=k
            )
            predictions_list.append(pred)
    
    if predictions_list:
        predictions_array = np.array(predictions_list)
        final_pred = []
        for i in range(predictions_array.shape[1]):
            values, counts = np.unique(predictions_array[:, i], return_counts=True)
            final_pred.append(values[np.argmax(counts)])
        
        unknown_count = np.sum(np.array(final_pred) == -1)
        unknown_ratio = unknown_count / len(final_pred) * 100
        
        k_results.append({
            'k': k,
            'unknown_count': unknown_count,
            'unknown_ratio': unknown_ratio
        })
        
        print(f"k={k:2d}: unknown={unknown_count:4d} ({unknown_ratio:5.2f}%)")

df_k_results = pd.DataFrame(k_results)
print("\n=== k-value tuning results ===")
print(df_k_results)

## 16. 結果をGoogle Driveにバックアップ

In [ ]:
# ★バックアップ先のパスを設定★
GDRIVE_BACKUP_PATH = "gdrive:atma22"

# モデル・ログ・提出ファイルをバックアップ
print("=== Backing up results to Google Drive ===")

!rclone sync /content/{PROJECT_NAME}/models {GDRIVE_BACKUP_PATH}/models --progress
!rclone sync /content/{PROJECT_NAME}/logs {GDRIVE_BACKUP_PATH}/logs --progress
!rclone sync /content/{PROJECT_NAME}/data/submission {GDRIVE_BACKUP_PATH}/data/submission --progress
!rclone sync /content/{PROJECT_NAME}/data/features {GDRIVE_BACKUP_PATH}/data/features --progress

print("\n✅ Backup completed!")

## 17. Git push（オプション）

In [ ]:
# ソースコードに変更がある場合はコミット
# ※大容量ファイル（models, data）はGitに含めない

%cd /content/{PROJECT_NAME}

# 状態確認
!git status

# コミット・プッシュ（必要に応じて実行）
# !git add src/ notebooks/
# !git commit -m "Add KNN experiment results from Colab"
# !git push

## まとめ

### ✅ 実行完了内容
1. GPU環境でKNN学習（高速化）
2. 5-fold CV評価
3. テストデータ予測
4. kチューニング実験
5. Google Driveへバックアップ

### 📊 次のステップ
- ローカルで結果を確認（rclone sync）
- Prototype法との比較
- EfficientNet-B0で再実験
- アンサンブル実装